In [9]:
import collections

import numpy as np

import util
import svm


def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
        The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    return message.lower().split()
    '''
    words = []
    start = 0
    for i in range(len(message)):
        if message[i] == ' ':
            word = message[start:i].lower()
            if word:  # skip empty
                words.append(word)
            start = i + 1
    if start < len(message):
        word = message[start:].lower()
        if word:
            words.append(word)
    return words'''
    # *** END CODE HERE ***


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message. 

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    word_count = collections.defaultdict(int)
    for message in messages:
        words = set(get_words(message))
        for word in words:
            word_count[word] += 1
    dictionary = {}
    index = 0
    for word, count in word_count.items():
        if count >= 5:
            dictionary[word] = index
            index += 1
    return dictionary
    # *** END CODE HERE ***


def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each 
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that 
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.
    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    matrix = np.zeros((len(messages), len(word_dictionary)))
    for i in range(len(messages)):
        words = get_words(messages[i])
        for word,idx in word_dictionary.items():
            matrix[i][idx] = words.count(word)
    return matrix
    # *** END CODE HERE ***




In [10]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    N, D = matrix.shape
    phi_y = np.mean(labels)

    X_pos = matrix[labels == 1, :]
    X_neg = matrix[labels == 0, :]

    phi_pos = []
    phi_neg = []
    alpha = 1

    # Multinomial counts
    count_j_pos = X_pos.sum(axis=0)
    count_j_neg = X_neg.sum(axis=0)

    
    total_tokens_pos = count_j_pos.sum()
    total_tokens_neg = count_j_neg.sum()

    alpha = 1   

    denom_pos = total_tokens_pos + alpha * D
    denom_neg = total_tokens_neg + alpha * D

    
    phi_pos = (count_j_pos + alpha) / denom_pos
    phi_neg = (count_j_neg + alpha) / denom_neg
    '''for j in range(D):
        sum_pos = 0
        sum_neg = 0
        for i in range(N):
            if labels[i] == 1:
                sum_pos += matrix[i][j]
            else:
                sum_neg += matrix[i][j]
        phi_pos.append((sum_pos + alpha)/(total_tokens_pos + alpha*D))
        phi_neg.append((sum_neg + alpha)/(total_tokens_neg + alpha*D))
    '''

    phi_pos = np.array(phi_pos)
    phi_neg = np.array(phi_neg)
    return {"phi_y" : phi_y,
            "phi_pos" : phi_pos,
            "phi_neg" : phi_neg
            }
    # *** END CODE HERE ***






In [11]:
def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    predictions = []
    phi_y = model["phi_y"]
    phi_pos = model["phi_pos"]
    phi_neg = model["phi_neg"]

    N,D = matrix.shape
    for i in range(N):
        A = np.log(phi_y)
        B = np.log(1-phi_y)
        for j in range(D):
            A += matrix[i][j]* np.log(phi_pos[j])
            B += matrix[i][j]* np.log(phi_neg[j])
        num = A
        denom = np.log(np.exp(A) + np.exp(B))
        predictions.append(np.exp(num - denom))

    return np.array(predictions)
    # *** END CODE HERE ***


In [12]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    phi_pos = model["phi_pos"]
    phi_neg = model["phi_neg"]
    phi_y = model["phi_y"]

    indicators = np.log(phi_pos / phi_neg)

    top_indices = np.argsort(indicators)[::-1][:5]

    # Create reverse dictionary: index -> word
    reverse_dict = {index: word for word, index in dictionary.items()}

    # Map indices to words
    top_words = [reverse_dict[i] for i in top_indices]

    return top_words
    # *** END CODE HERE ***




In [13]:
def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider
    
    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***
    best_sigma = None
    best_accuracy = 0

    for sigma in radius_to_consider:
        model_predictions = svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, sigma)
        accuracy = np.mean(model_predictions == val_labels)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_sigma = sigma

    return best_sigma
    # *** END CODE HERE ***




In [14]:
def main():
    train_messages, train_labels = util.load_spam_dataset('../data/ds6_train.tsv')
    val_messages, val_labels = util.load_spam_dataset('../data/ds6_val.tsv')
    test_messages, test_labels = util.load_spam_dataset('../data/ds6_test.tsv')
    
    dictionary = create_dictionary(train_messages)

    util.write_json('./output/p06_dictionary', dictionary)

    train_matrix = transform_text(train_messages, dictionary)

    np.savetxt('./output/p06_sample_train_matrix', train_matrix[:100,:])

    val_matrix = transform_text(val_messages, dictionary)
    test_matrix = transform_text(test_messages, dictionary)

    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

    np.savetxt('./output/p06_naive_bayes_predictions', naive_bayes_predictions)

    naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

    print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

    top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

    print('The top 5 indicative words for Naive Bayes are: ', top_5_words)

    util.write_json('./output/p06_top_indicative_words', top_5_words)

    optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])

    util.write_json('./output/p06_optimal_radius', optimal_radius)

    print('The optimal SVM radius was {}'.format(optimal_radius))

    svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)

    svm_accuracy = np.mean(svm_predictions == test_labels)

    print('The SVM model had an accuracy of {} on the testing set'.format(svm_accuracy, optimal_radius))


if __name__ == "__main__":
    main()


Naive Bayes had an accuracy of 0.044802867383512544 on the testing set
The top 5 indicative words for Naive Bayes are:  ['claim', 'won', 'prize', 'tone', 'urgent!']
The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9695340501792115 on the testing set
